In [1]:
import sklearn
import numpy as np
import pandas as pd
from dev import DEVALGO 
from smote import SMOTEBoost
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imblearn.metrics import geometric_mean_score

In [2]:
MINORITY = 1
MAJORITY = 0
CLASS_NAME = "class"

In [3]:
def pprint_dict(d):
    for key, value in d.items():
        print(key + ": " + "{0:.2f}".format(value/100.0))

In [4]:
df = pd.read_csv("data/Vowel/vowel.csv").sample(frac=1, random_state=0).reset_index(drop=True)
df.head()

,TT,SpeakerNumber,Sex,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,class
0,1,11,0,-3.265,1.933,-0.316,0.602,0.438,0.336,0.048,0.134,-0.160,-0.991,10
1,1,9,0,-4.753,3.421,-0.189,0.656,-0.984,1.443,-0.652,0.546,0.123,-0.326,9
2,0,4,1,-3.049,0.084,0.981,1.193,0.761,0.495,-0.714,-0.466,-0.568,-0.149,1
3,1,11,0,-4.028,0.982,-0.648,1.478,-0.048,2.108,-0.146,0.467,-0.654,-0.193,9
4,0,0,0,-3.046,0.387,-1.165,0.940,-0.386,1.978,-0.186,0.811,-0.394,-0.198,1


In [5]:
# Data transformation

# # Uncomment this when testing on abalone
# df = df[(df["class"] == 18) | (df["class"] == 9)].reset_index(drop=True)
# df["class"].replace({18:1, 9:0}, inplace=True)
# df.head()

# Uncomment this when testing on vowel
df["class"].replace({0:1, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0}, inplace=True)
df.head()

,TT,SpeakerNumber,Sex,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,class
0,1,11,0,-3.265,1.933,-0.316,0.602,0.438,0.336,0.048,0.134,-0.160,-0.991,0
1,1,9,0,-4.753,3.421,-0.189,0.656,-0.984,1.443,-0.652,0.546,0.123,-0.326,0
2,0,4,1,-3.049,0.084,0.981,1.193,0.761,0.495,-0.714,-0.466,-0.568,-0.149,0
3,1,11,0,-4.028,0.982,-0.648,1.478,-0.048,2.108,-0.146,0.467,-0.654,-0.193,0
4,0,0,0,-3.046,0.387,-1.165,0.940,-0.386,1.978,-0.186,0.811,-0.394,-0.198,0


In [6]:
# Outcomes distribution

df["class"].value_counts()

0    900
1     90
Name: class, dtype: int64

In [7]:
# Calculate imbalanced ratio
frq = df["class"].value_counts()
ratio = max(frq)/min(frq)
"{0:.1f}".format(ratio)

'10.0'

# Use the following for testing Decision tree

- run 100 iterations
- 50/50 training/testing
- Decision tree

In [8]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    clf_tree = DecisionTreeClassifier(random_state=0)
    clf_tree.fit(X_train, y_train)
    y_pred = clf_tree.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["weighted avg"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["weighted avg"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["weighted avg"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

In [9]:
pprint_dict(res)

precision: 0.96
recall: 0.96
f1-score: 0.96
specificity: 0.99
sensitivity: 0.66
overall accuracy: 0.96
auc: 0.83
g_mean: 0.81


# Use the following for testing SMOTE

- run 100 iterations
- 50/50 training/testing
- Decision tree
- N = 200

In [10]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    if i == 0:
        print("Shape of X_train before oversampling: " + str(X_train.shape))
        print("Outcome distribution of X_train before oversampling: " + str(np.bincount(y_train)))
    # Oversample training data
    sm = SMOTE(random_state=0)
    sm.fit(X_train, y_train)
    X_train_r, y_train_r = sm.sample(X_train, y_train)
    if i == 0:
        print("Shape of X_train after oversampling: " + str(X_train_r.shape))
        print("Outcome distribution of X_train after oversampling: " + str(np.bincount(y_train_r)))
    # Build classifier on resampled data
    clf_tree = DecisionTreeClassifier(random_state=0)
    clf_tree.fit(X_train_r, y_train_r)
    y_pred = clf_tree.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["weighted avg"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["weighted avg"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["weighted avg"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

Shape of X_train before oversampling: (495, 13)
Outcome distribution of X_train before oversampling: [452  43]
Shape of X_train after oversampling: (904, 13)
Outcome distribution of X_train after oversampling: [452 452]


In [11]:
pprint_dict(res)

precision: 0.95
recall: 0.95
f1-score: 0.95
specificity: 0.97
sensitivity: 0.79
overall accuracy: 0.95
auc: 0.88
g_mean: 0.87


# Use the following for testing ADASYN

- run 100 iterations
- 50/50 training/testing
- Decision tree
- A fully balanced dataset after synthesizing
- Dth = 0.75 (Dth is a preset threshold for the maximum tolerated degree of class imbalance ratio)

In [12]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    if i == 0:
        print("Shape of X_train before oversampling: " + str(X_train.shape))
        print("Outcome distribution of X_train before oversampling: " + str(np.bincount(y_train)))
    # Oversample training data
    ada = ADASYN(random_state=0)
    ada.fit(X_train, y_train)
    X_train_r, y_train_r = ada.sample(X_train, y_train)
    if i == 0:
        print("Shape of X_train after oversampling: " + str(X_train_r.shape))
        print("Outcome distribution of X_train after oversampling: " + str(np.bincount(y_train_r)))
    # Build classifier on resampled data
    clf_tree = DecisionTreeClassifier(random_state=0)
    clf_tree.fit(X_train_r, y_train_r)
    y_pred = clf_tree.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["weighted avg"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["weighted avg"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["weighted avg"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

Shape of X_train before oversampling: (495, 13)
Outcome distribution of X_train before oversampling: [452  43]
Shape of X_train after oversampling: (893, 13)
Outcome distribution of X_train after oversampling: [452 441]


In [13]:
pprint_dict(res)

precision: 0.96
recall: 0.95
f1-score: 0.95
specificity: 0.97
sensitivity: 0.83
overall accuracy: 0.95
auc: 0.90
g_mean: 0.90


# Use the following for testing SMOTEBoost

- run 100 iterations
- 50/50 training/testing
- Decision tree

In [14]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    clf1 = SMOTEBoost(random_state=0)
    clf1.fit(X_train, y_train)
    y_pred = clf1.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["weighted avg"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["weighted avg"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["weighted avg"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

In [15]:
pprint_dict(res)

precision: 0.98
recall: 0.97
f1-score: 0.97
specificity: 0.98
sensitivity: 0.96
overall accuracy: 0.97
auc: 0.97
g_mean: 0.97


# Use the following for testing Dev_algo

- run 100 iterations
- 50/50 training/testing
- Decision tree

In [16]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    clf1 = DEVALGO(random_state=0, n_samples=319)
    clf1.fit(X_train, y_train)
    y_pred = clf1.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["weighted avg"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["weighted avg"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["weighted avg"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

In [17]:
 pprint_dict(res)

precision: 0.99
recall: 0.99
f1-score: 0.99
specificity: 0.99
sensitivity: 0.98
overall accuracy: 0.99
auc: 0.98
g_mean: 0.98
